In [2]:
# !pip install albumentations
# !pip install tensorboard
# !pip install effdet
# !pip install datasets
# !pip install albumentations
# !pip install effdet
# !pip install tensorboard
# !pip install pytorch-lightning

In [1]:
from pathlib import Path
from PIL import Image
import os
import numpy as np
import torch
from torch.utils.data import Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Subset

/home/jian6768/anaconda3/envs/ffdenv2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# """
# This snippet of code downloads data from pyronear and creates the appropriate directory structure.

# """
# import os
# from datasets import load_dataset

# # Define paths
# REPO_ID = "pyronear/pyro-sdis"
# OUTPUT_DIR = "./pyro-sdis"
# IMAGE_DIR = os.path.join(OUTPUT_DIR, "images")
# LABEL_DIR = IMAGE_DIR.replace("images", "labels")

# # Create the directory structure
# for split in ["train", "val"]:
#     os.makedirs(os.path.join(IMAGE_DIR, split), exist_ok=True)
#     os.makedirs(os.path.join(LABEL_DIR, split), exist_ok=True)

# # Load the dataset from the Hugging Face Hub
# dataset = load_dataset(REPO_ID)

# # Save images and labels to disk
# for split in ["train", "val"]:
#     split_data = dataset[split]
#     # split_data = split_data.select(range(1000))
#     for example in split_data:
#         # Extract fields
#         image = example["image"]
#         annotations = example["annotations"]
#         image_name = example["image_name"]

#         # Save image
#         image_path = os.path.join(IMAGE_DIR, split, image_name)
#         image.save(image_path)

#         # Save annotations (labels)
#         label_path = os.path.join(LABEL_DIR, split, os.path.splitext(image_name)[0] + ".txt")
#         with open(label_path, "w") as f:
#             f.write(annotations)

# print(f"Dataset saved to {OUTPUT_DIR}")


README.md:   0%|          | 0.00/7.37k [00:00<?, ?B/s]

train-00000-of-00006.parquet:   0%|          | 0.00/481M [00:00<?, ?B/s]

train-00001-of-00006.parquet:   0%|          | 0.00/485M [00:00<?, ?B/s]

train-00002-of-00006.parquet:   0%|          | 0.00/482M [00:00<?, ?B/s]

train-00003-of-00006.parquet:   0%|          | 0.00/483M [00:00<?, ?B/s]

...

train-00005-of-00006.parquet:   0%|          | 0.00/483M [00:00<?, ?B/s]

val-00000-of-00001.parquet:   0%|          | 0.00/390M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/29537 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/4099 [00:00<?, ? examples/s]

Dataset saved to ./pyro-sdis


In [4]:
#check number of cpu cores available. Will be used to set the number of workers.
#Rule of thumb is to set min(4,os.cpu_count()-1)
import os
print(os.cpu_count())

32


In [2]:
def get_train_transforms(target_img_size=640):
    return A.Compose(
        [
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.2),
            # simulates when the camera is moving quickly.
            A.MotionBlur(p=0.1),
            # basic blurring
            A.Blur(blur_limit=3, p=0.1),
            A.ColorJitter(p=0.1),
            A.RandomFog(p=0.1),
            
            A.Resize(height=target_img_size, width=target_img_size, p=1),
            A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ToTensorV2(p=1),
        ],
        p=1.0,
        bbox_params=A.BboxParams(
            format="pascal_voc", min_area=0, min_visibility=0, label_fields=["labels"]
        ),
    )


def get_valid_transforms(target_img_size=640):
    return A.Compose(
        [
            A.Resize(height=target_img_size, width=target_img_size, p=1),
            A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ToTensorV2(p=1),
        ],
        p=1.0,
        bbox_params=A.BboxParams(
            format="pascal_voc", min_area=0, min_visibility=0, label_fields=["labels"]
        ),
    )

In [3]:

#Image directory path would be /pyro-sdis/images/train
class SmokeDataset(Dataset):

    def __init__(self, root_dir, split='train', transforms=get_valid_transforms(), class_offset=0):
        self.images_dir = os.path.join(root_dir, 'images', split)
        self.labels_dir = os.path.join(root_dir, 'labels', split)
        self.image_filenames = sorted([f for f in os.listdir(self.images_dir) if f.endswith(('.jpg', '.png'))])
        self.transforms = transforms
        self.class_offset = class_offset

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, index):
        # image_name = self.images[index]
        # print(f"[__getitem__] Loading index: {index}")
        image_name = self.image_filenames[index]
        image_path = os.path.join(self.images_dir, image_name)
        label_path = os.path.join(self.labels_dir, os.path.splitext(image_name)[0] + '.txt')

        image = np.array(Image.open(image_path).convert("RGB"))
        H, W, _ = image.shape

        image_id = index


        boxes = []

        # image = PIL.Image.open(os.path.join(self.images_dir, image_name))
        #In YOLO, each image will be associated with a text file. The text file may have any number of bounding boxes. If zero, the txt file will be empty.
        #Convert data from pascal to yolo format.
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    cls, x_c, y_c, w, h = map(float, line.strip().split())

                    x_c *= W
                    y_c *= H
                    w *= W
                    h *= H
                    # print("H",H)
                    # print("W",W)
                    # print("x_c: ",x_c)
                    # print("y_c: ",y_c)
                    # print("w:",w)
                    # print("h:",h)

                    # x_min = x_c - w / 2
                    # y_min = y_c - h / 2
                    # x_max = x_c + w / 2
                    # y_max = y_c + h / 2
                    x_min = max(int(x_c - w / 2), 0)
                    y_min = max(int(y_c - h / 2), 0)
                    #-1 because image indexing is zero based. the last pixel would be W - 1
                    x_max = min(int(x_c + w / 2), W - 1)
                    y_max = min(int(y_c + h / 2), H - 1)

                    boxes.append([x_min, y_min, x_max, y_max])
                    # labels.append(int(cls) + self.class_offset)

        #Question. If one image has no associated bounding box, how should it be expressed in pasccal form?

        #Models typically expect shape of (N,4)
        if len(boxes) == 0:
            boxes = np.empty((0,4))

        boxes = np.array(boxes)
        # labels = np.array(labels, dtype=np.int64)


        pascal_bboxes = boxes

        class_labels = np.ones(len(pascal_bboxes))

        # return image, pascal_bboxes, class_labels, index

        sample = {
            # "image": np.array(image, dtype=np.float32),
            "image": np.array(image, dtype=np.float32),
            "bboxes": pascal_bboxes,
            "labels": class_labels,
        }

        sample = self.transforms(**sample)
        sample["bboxes"] = np.array(sample["bboxes"])
        image = sample["image"]
        labels = sample["labels"]

        _, new_h, new_w = image.shape
        sample["bboxes"][:, [0, 1, 2, 3]] = sample["bboxes"][
            :, [1, 0, 3, 2]
        ]  # convert to yxyx


        target = {
            "bbox": torch.as_tensor(sample["bboxes"], dtype=torch.float32),
            "cls": torch.as_tensor(labels),
            "image_id": torch.tensor([image_id]),
            "img_size": (new_h, new_w),

            # "img_size": torch.tensor((new_h, new_w)),
            "img_scale": torch.tensor([1.0]),
        }

        image_id = index
        return image, target, image_id




In [4]:
batch_size = 8

# train_adaptor = SmokeDatasetAdaptor(root_dir='./pyro-sdis', split='train')
train_dataset = SmokeDataset(root_dir='./pyro-sdis', split='train', transforms=get_train_transforms())
val_dataset = SmokeDataset(root_dir='./pyro-sdis', split='val', transforms=get_valid_transforms())


# subset_indices = list(range(5000))
# subset_indices_val = list(range(1000))


# train_dataset = Subset(train_dataset, subset_indices)
# val_dataset = Subset(val_dataset, subset_indices_val)


In [6]:
#V1 removing image ids
def collate_fn(batch):
        images, targets, image_ids = tuple(zip(*batch))
        images = torch.stack(images)
        images = images.float()

        boxes = [target["bbox"].float() for target in targets]
        labels = [target["cls"].float() for target in targets]
        # boxes = torch.stack([target["bbox"].float() for target in targets])
        # labels = torch.stack([target["cls"].float() for target in targets])
        img_size = torch.tensor([target["img_size"] for target in targets]).float()
        # img_size = torch.tensor([target["img_size"] for target in targets])
        img_scale = torch.tensor([target["img_scale"] for target in targets]).float()

        annotations = {
            "bbox": boxes,
            "cls": labels,
            "img_size": img_size,
            "img_scale": img_scale,
        }

        return images, annotations, targets, image_ids

In [7]:
# train_dataloader = torch.utils.data.DataLoader(
#     train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, collate_fn=collate_fn)
num_workers = 4
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn=collate_fn)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=collate_fn)

In [8]:
data_iter = iter(train_dataloader)
sample_batch = next(data_iter)

print(f"Batch type: {type(sample_batch)}")
if isinstance(sample_batch, (tuple, list)):
    for i, item in enumerate(sample_batch):
        print(f"Item {i}: type={type(item)}, dtype={item.dtype if isinstance(item, torch.Tensor) else 'N/A'}")
else:
    print(f"Single item: type={type(sample_batch)}, dtype={sample_batch.dtype if isinstance(sample_batch, torch.Tensor) else 'N/A'}")

Batch type: <class 'tuple'>
Item 0: type=<class 'torch.Tensor'>, dtype=torch.float32
Item 1: type=<class 'dict'>, dtype=N/A
Item 2: type=<class 'tuple'>, dtype=N/A
Item 3: type=<class 'tuple'>, dtype=N/A


In [9]:
import timm
from effdet.config.model_config import efficientdet_model_param_dict
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain, DetBenchPredict
from effdet.efficientdet import HeadNet

timm.list_models("efficientdet_d1")

# num_classes = 1
# model_name = "efficientdet_d0"

# # torch.tensor()
# image_size = (512,512)
# config = get_efficientdet_config(model_name)
# config.update({'num_classes': num_classes})
# config.update({'image_size': image_size})

# print(config)

# net = EfficientDet(config, pretrained_backbone=True)
# net.class_net = HeadNet(
#     config,
#     num_outputs=config.num_classes,
# )

# #Essentially, DetBenchTrain will take in images and targets from the dataloader.
# model = DetBenchTrain(net, config)


[]

In [ ]:
import torch
from pytorch_lightning import LightningModule
# from pytorch_lightning.core.decorators import auto_move_data
from torchmetrics.detection.mean_ap import MeanAveragePrecision



class EfficientDetModel(LightningModule):
    def __init__(
        self,
        num_classes=1,
        img_size=512,
        prediction_confidence_threshold=0.2,
        learning_rate=0.0002,
        wbf_iou_threshold=0.44,
        inference_transforms=get_valid_transforms(target_img_size=512),
        #Edit here.
        model_architecture="efficientdet_d1",
        batch_size=8
    ):
        super().__init__()
        self.img_size = img_size

        #KJ added:
        self.map_metric = MeanAveragePrecision()
        
        self.batch_size = batch_size

        #KJ added


        image_size = (img_size,img_size)
        config = get_efficientdet_config(model_architecture)
        config.update({'num_classes': num_classes})
        config.update({'image_size': image_size})

        print(config)

        net = EfficientDet(config, pretrained_backbone=True)
        net.class_net = HeadNet(
            config,
            num_outputs=config.num_classes,
        )

        #Essentially, DetBenchTrain will take in images and targets from the dataloader.
        model = DetBenchTrain(net, config)



        self.model = model
        self.prediction_confidence_threshold = prediction_confidence_threshold
        self.lr = learning_rate
        self.wbf_iou_threshold = wbf_iou_threshold
        self.inference_tfms = inference_transforms



    def forward(self, images, targets):
        return self.model(images, targets)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.model.parameters(), lr=self.lr)


    def training_step(self, batch, batch_idx):
        images, annotations, _, image_ids = batch

        losses = self.model(images, annotations)


        logging_losses = {
            "class_loss": losses["class_loss"].detach(),
            "box_loss": losses["box_loss"].detach(),
        }

        self.log("train_loss", losses["loss"], on_step=True, on_epoch=True, prog_bar=True,
                 logger=True, batch_size=batch_size)
        self.log(
            "train_class_loss", losses["class_loss"], on_step=True, on_epoch=True, prog_bar=True,
            logger=True, batch_size=batch_size
        )
        self.log("train_box_loss", losses["box_loss"], on_step=True, on_epoch=True, prog_bar=True,
                 logger=True, batch_size=batch_size)

        return losses['loss']


    @torch.no_grad()
    def validation_step(self, batch, batch_idx):
        #What that gets spitted out from data loader.
        images, annotations, targets, image_ids = batch
        outputs = self.model(images, annotations)

        detections = outputs["detections"]

        #Code for MAP; Torchmetrics MAP expects predictions and targets/annotations to be in xmin, ymin, xmax, ymax format
        #Code for MAP - setting up predictions
        predictions_map = []

        #iterate through batch size number of times
        for i in range(detections.shape[0]):
            sample_data = {}
            pred_bboxes = detections[i][:,:4]
            pred_labels = detections[i][:,-1]
            pred_scores = detections[i][:,-2]
            sample_data["boxes"] = pred_bboxes
            sample_data["labels"] = pred_labels.to(torch.int64)
            sample_data["scores"] = pred_scores
            predictions_map.append(sample_data)
        # print(predictions_map)


        #Code for MAP - setting up annotations. Note that annotations from dataloader are in the form of yxyx rather than xyxy.
        annotations_map = []
        

        for i in range(len(annotations["bbox"])):
            sample_data = {}

            # sample_data["boxes"] = annotations["bbox"][i]
            # #convert annotation from yxyx to xyxy format to utilize torchmetrics MAP.
            # sample_data["boxes"][:, [0, 1, 2, 3]] = sample_data["boxes"][:, [1, 0, 3, 2]]

            boxes_xyxy = annotations["bbox"][i].clone()
            boxes_xyxy[:, [0, 1, 2, 3]] = boxes_xyxy[:, [1, 0, 3, 2]]
            sample_data["boxes"] = boxes_xyxy

            sample_data["labels"] = annotations["cls"][i].to(torch.int64)
            annotations_map.append(sample_data)

            

        # print(annotations_map)

        self.map_metric.update(predictions_map, annotations_map)
        


        #Code for losses
        batch_predictions = {
            "predictions": detections,
            "targets": targets,
            "image_ids": image_ids,
        }

        logging_losses = {
            "class_loss": outputs["class_loss"].detach(),
            "box_loss": outputs["box_loss"].detach(),
        }

        self.log("valid_loss", outputs["loss"], on_step=True, on_epoch=True, prog_bar=True,
                 logger=True, sync_dist=True, batch_size=batch_size)
        self.log(
            "valid_class_loss", logging_losses["class_loss"], on_step=True, on_epoch=True,
            prog_bar=True, logger=True, sync_dist=True, batch_size=batch_size
        )
        self.log("valid_box_loss", logging_losses["box_loss"], on_step=True, on_epoch=True,
                 prog_bar=True, logger=True, sync_dist=True, batch_size=batch_size)

        return {'loss': outputs["loss"], 'batch_predictions': batch_predictions}


    def on_validation_epoch_end(self):

        #KJ added
        #self.map_metric compute returns a map_dictionary
        #https://lightning.ai/docs/torchmetrics/stable/detection/mean_average_precision.html
        #types of map keys available in documentation above.
        map_metrics = self.map_metric.compute()

        self.log("mAP", map_metrics["map"], prog_bar=True, logger=True,batch_size=batch_size)
        self.log("mAP_50", map_metrics["map_50"], prog_bar=True, logger=True, batch_size=batch_size)
        self.map_metric.reset()

        

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        images, _, _, image_ids = batch

        # Forward pass through the model to get raw predictions
        outputs = self.model(images, None)
        detections = outputs["detections"]

        # Prepare the predictions in the desired format
        predictions = []
        for i in range(detections.shape[0]):
            pred_bboxes = detections[i][:, :4]
            pred_scores = detections[i][:, -2]
            pred_labels = detections[i][:, -1].int()
            predictions.append({
                "image_id": image_ids[i],
                "boxes": pred_bboxes,
                "scores": pred_scores,
                "labels": pred_labels
            })

        return predictions

    #Potentially something to implement.
    # def predict_step(self, batch, batch_idx, dataloader_idx=0):
    #     images, _, _, image_ids = batch
    #     outputs = self.model(images, None)
    #     detections = outputs["detections"]
    #     return detections

In [13]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

# checkpoint_callback = ModelCheckpoint(
#     monitor="valid_loss",             # metric to track
#     dirpath="checkpoints/",           # custom save path
#     filename="effdet-{epoch:02d}-{valid_loss:.2f}",  # filename format
#     save_top_k=1,                     # only keep best model
#     mode="min"                        # lower is better
# )

# trainer = Trainer(
#     max_epochs=10,
#     callbacks=[checkpoint_callback],
#     accelerator="gpu",
#     devices=[0]
# )

trainer = Trainer(accelerator='gpu', devices=[0], max_epochs=40, num_sanity_val_steps=1)
effdetmodel = EfficientDetModel(batch_size=batch_size)
trainer.fit(effdetmodel,train_dataloader,val_dataloader)

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


{'name': 'efficientdet_d0', 'backbone_name': 'efficientnet_b0', 'backbone_args': {'drop_path_rate': 0.1}, 'backbone_indices': None, 'image_size': [512, 512], 'num_classes': 1, 'min_level': 3, 'max_level': 7, 'num_levels': 5, 'num_scales': 3, 'aspect_ratios': [[1.0, 1.0], [1.4, 0.7], [0.7, 1.4]], 'anchor_scale': 4.0, 'pad_type': '', 'act_type': 'swish', 'norm_layer': None, 'norm_kwargs': {'eps': 0.001, 'momentum': 0.01}, 'box_class_repeats': 3, 'fpn_cell_repeats': 3, 'fpn_channels': 64, 'separable_conv': True, 'apply_resample_bn': True, 'conv_bn_relu_pattern': False, 'downsample_type': 'max', 'upsample_type': 'nearest', 'redundant_bias': False, 'head_bn_level_first': False, 'head_act_type': None, 'fpn_name': None, 'fpn_config': None, 'fpn_drop_path_rate': 0.0, 'alpha': 0.25, 'gamma': 1.5, 'label_smoothing': 0.0, 'legacy_focal': False, 'jit_loss': False, 'delta': 0.1, 'box_loss_weight': 50.0, 'soft_nms': False, 'max_detection_points': 5000, 'max_det_per_image': 100, 'url': 'https://githu

Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.
You are using a CUDA device ('NVIDIA GeForce RTX 4070 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                 | Params | Mode 
---------------------------------------------------------------
0 | map_metric    | MeanAveragePrecision | 0      | train
1 | map_metric_v2 | MeanAveragePrecision | 0      | train
2 | model         | DetBenchTrain        | 3.8 M  | train
---------------------------------------------

Epoch 39: 100%|██████████| 3693/3693 [17:07<00:00,  3.59it/s, v_num=2, train_loss_step=0.624, train_class_loss_step=0.345, train_box_loss_step=0.00558, valid_loss_step=2.470, valid_class_loss_step=1.750, valid_box_loss_step=0.0144, valid_loss_epoch=0.851, valid_class_loss_epoch=0.606, valid_box_loss_epoch=0.0049, mAP=0.252, mAP_50=0.593, mAP_v2=9.52e-6, mAP_50_v2=3.63e-5, train_loss_epoch=0.636, train_class_loss_epoch=0.394, train_box_loss_epoch=0.00484]    

`Trainer.fit` stopped: `max_epochs=40` reached.


Epoch 39: 100%|██████████| 3693/3693 [17:07<00:00,  3.59it/s, v_num=2, train_loss_step=0.624, train_class_loss_step=0.345, train_box_loss_step=0.00558, valid_loss_step=2.470, valid_class_loss_step=1.750, valid_box_loss_step=0.0144, valid_loss_epoch=0.851, valid_class_loss_epoch=0.606, valid_box_loss_epoch=0.0049, mAP=0.252, mAP_50=0.593, mAP_v2=9.52e-6, mAP_50_v2=3.63e-5, train_loss_epoch=0.636, train_class_loss_epoch=0.394, train_box_loss_epoch=0.00484]


In [14]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

effdetmodel = EfficientDetModel()

#Resume training without load
ckpt_path = "/content/lightning_logs/version_0/checkpoints/epoch=9-step=12310.ckpt"
trainer = Trainer(accelerator='gpu', devices=[0], max_epochs=20, num_sanity_val_steps=1)
trainer.fit(effdetmodel,train_dataloader,val_dataloader, ckpt_path=ckpt_path)

{'name': 'efficientdet_d0', 'backbone_name': 'efficientnet_b0', 'backbone_args': {'drop_path_rate': 0.1}, 'backbone_indices': None, 'image_size': [512, 512], 'num_classes': 1, 'min_level': 3, 'max_level': 7, 'num_levels': 5, 'num_scales': 3, 'aspect_ratios': [[1.0, 1.0], [1.4, 0.7], [0.7, 1.4]], 'anchor_scale': 4.0, 'pad_type': '', 'act_type': 'swish', 'norm_layer': None, 'norm_kwargs': {'eps': 0.001, 'momentum': 0.01}, 'box_class_repeats': 3, 'fpn_cell_repeats': 3, 'fpn_channels': 64, 'separable_conv': True, 'apply_resample_bn': True, 'conv_bn_relu_pattern': False, 'downsample_type': 'max', 'upsample_type': 'nearest', 'redundant_bias': False, 'head_bn_level_first': False, 'head_act_type': None, 'fpn_name': None, 'fpn_config': None, 'fpn_drop_path_rate': 0.0, 'alpha': 0.25, 'gamma': 1.5, 'label_smoothing': 0.0, 'legacy_focal': False, 'jit_loss': False, 'delta': 0.1, 'box_loss_weight': 50.0, 'soft_nms': False, 'max_detection_points': 5000, 'max_det_per_image': 100, 'url': 'https://githu

model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_0/checkpoints/epoch=9-step=12310.ckpt
/usr/local/lib/python3.11/di

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [15]:
import shutil
from google.colab import files

shutil.make_archive('lightning_logs',"zip","lightning_logs")


'/content/lightning_logs.zip'

In [16]:

files.download('lightning_logs.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#To run tensorboard in bash.
!tensorboard --logdir lightning_logs/

2025-04-06 07:48:25.600422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743925705.621604   53128 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743925705.628154   53128 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.18.0 at http://localhost:6006/ (Press CTRL+C to quit)


In [ ]:
#Prediction method to be implemented.

# Initialize your model
model = EfficientDetModel.load_from_checkpoint("path/to/your/checkpoint.ckpt")

# Set up the Trainer
trainer = Trainer(accelerator='gpu', devices=[0])

# Perform prediction
predictions = trainer.predict(model, dataloaders=predict_dataloader)

In [ ]:


checkpoint_callback = ModelCheckpoint(
    monitor="valid_loss",             # metric to track
    dirpath="checkpoints/",           # custom save path
    filename="effdet-{epoch:02d}-{valid_loss:.2f}",  # filename format
    save_top_k=1,                     # only keep best model
    mode="min"                        # lower is better
)

In [ ]:
model = EfficientDetModel.load_from_checkpoint(

    "lightning_logs/version_11/checkpoints/epoch=4-step=18465.ckpt"
)
model.eval()



In [ ]:

# model.to(device)
# for images, annotations, targets, image_ids in train_dataloader:
#     print(images)
#     # print(images)
#     # print(annotations)

#     images = torch.stack([image.to(device) for image in images])

#     annotations = {k: [t.to(device) for t in v] for k,v in annotations.items()}
#     loss_dict = model(images, annotations)
#     break


In [ ]:
import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.transforms import functional as F
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Set up the Adam optimizer
learning_rate = 0.001  # Adjust as needed
beta1 = 0.9            # Default value for beta1
beta2 = 0.999          # Default value for beta2
epsilon = 1e-8         # Default value for epsilon
weight_decay = 0.0001  # Optional L2 regularization

#model.parameters have been change to params
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
    betas=(beta1, beta2),
    eps=epsilon,
    weight_decay=weight_decay
)


In [ ]:
#V4
import os
import torch
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from torchmetrics import Precision, Recall, F1Score

# Define checkpoint directory

checkpoint_dir = "./checkpoints/effdet"
checkpoint_dir_dbt = "./checkpoints/effdet/dbt"
# checkpoint_dir = "/content/drive/MyDrive/CVProject/manual_checkpoints/Finetuned"

os.makedirs(checkpoint_dir, exist_ok=True)

# Initialize metrics
# precision_metric = Precision(task="binary", average="macro")
# recall_metric = Recall(task="binary", average="macro")
# f1_metric = F1Score(task="binary", average="macro")
map_metric = MeanAveragePrecision(iou_type="bbox")

In [ ]:
for images, annotations, targets, image_ids in val_dataloader:
    annotations_map = []
    print(annotations)
    sample_data = {}

    for i in range(len(annotations["bbox"])):



    break

{'bbox': [tensor([], size=(0, 4)), tensor([], size=(0, 4)), tensor([[292.9778, 200.8000, 306.4889, 208.0000]]), tensor([[292.9778, 200.8000, 306.4889, 208.0000]]), tensor([[287.2889, 200.4000, 305.7778, 208.0000]]), tensor([[285.8667, 201.6000, 306.4889, 209.2000]]), tensor([[270.9333, 198.8000, 306.4889, 213.6000]]), tensor([[287.2889, 196.8000, 305.7778, 209.2000]])], 'cls': [tensor([]), tensor([]), tensor([1.]), tensor([1.]), tensor([1.]), tensor([1.]), tensor([1.]), tensor([1.])], 'img_size': tensor([[512., 512.],
        [512., 512.],
        [512., 512.],
        [512., 512.],
        [512., 512.],
        [512., 512.],
        [512., 512.],
        [512., 512.]]), 'img_scale': tensor([1., 1., 1., 1., 1., 1., 1., 1.])}


In [ ]:
# # net.eval()
# # net.to("cpu")
# # # model_predict = DetBenchTrain(net, config)
# # model_predict = DetBenchTrain(net)
# # model_predict.eval()
# model.eval()
# with torch.no_grad():
#     for images, annotations, targets, image_ids in val_dataloader:
#         if images is None or annotations is None:
#             continue
#         print(images)
#         print(images.shape)

#         predictions_map = []
#         predictions = model(images, annotations)
#         # predictions = model_predict(images, annotations)
#         # print("predictions shape", predictions.shape)
#         print("predictions here",predictions)
#         print(predictions['detections'].shape)
#         sample_data = {}

#         #iterate through batch size number of times
#         for i in range(predictions["detections"].shape[0]):

#             pred_bboxes = predictions["detections"][i][:,:4]
#             pred_labels = predictions["detections"][i][:,-1]
#             pred_scores = predictions["detections"][i][:,-2]
#             sample_data["boxes"] = pred_bboxes
#             sample_data["labels"] = pred_labels
#             sample_data["scores"] = pred_scores
#             predictions_map.append(sample_data)

#         print(predictions_map)

#         # print(len(predictions))
#         # print(predictions[0].shape)
#         # print("targets:")
#         # print(targets)
#         # print("annotations:")
#         # print(annotations)
#         break

In [ ]:
targets

({'bbox': tensor([], size=(0, 4)),
  'cls': tensor([]),
  'image_id': tensor([0]),
  'img_size': (512, 512),
  'img_scale': tensor([1.])},
 {'bbox': tensor([], size=(0, 4)),
  'cls': tensor([]),
  'image_id': tensor([1]),
  'img_size': (512, 512),
  'img_scale': tensor([1.])},
 {'bbox': tensor([[292.9778, 200.8000, 306.4889, 208.0000]]),
  'cls': tensor([1.], dtype=torch.float64),
  'image_id': tensor([2]),
  'img_size': (512, 512),
  'img_scale': tensor([1.])},
 {'bbox': tensor([[292.9778, 200.8000, 306.4889, 208.0000]]),
  'cls': tensor([1.], dtype=torch.float64),
  'image_id': tensor([3]),
  'img_size': (512, 512),
  'img_scale': tensor([1.])},
 {'bbox': tensor([[287.2889, 200.4000, 305.7778, 208.0000]]),
  'cls': tensor([1.], dtype=torch.float64),
  'image_id': tensor([4]),
  'img_size': (512, 512),
  'img_scale': tensor([1.])},
 {'bbox': tensor([[285.8667, 201.6000, 306.4889, 209.2000]]),
  'cls': tensor([1.], dtype=torch.float64),
  'image_id': tensor([5]),
  'img_size': (512, 51

In [ ]:
annotations

{'bbox': [tensor([], size=(0, 4)),
  tensor([], size=(0, 4)),
  tensor([[292.9778, 200.8000, 306.4889, 208.0000]]),
  tensor([[292.9778, 200.8000, 306.4889, 208.0000]]),
  tensor([[287.2889, 200.4000, 305.7778, 208.0000]]),
  tensor([[285.8667, 201.6000, 306.4889, 209.2000]]),
  tensor([[270.9333, 198.8000, 306.4889, 213.6000]]),
  tensor([[287.2889, 196.8000, 305.7778, 209.2000]])],
 'cls': [tensor([]),
  tensor([]),
  tensor([1.]),
  tensor([1.]),
  tensor([1.]),
  tensor([1.]),
  tensor([1.]),
  tensor([1.])],
 'img_size': tensor([[512., 512.],
         [512., 512.],
         [512., 512.],
         [512., 512.],
         [512., 512.],
         [512., 512.],
         [512., 512.],
         [512., 512.]]),
 'img_scale': tensor([1., 1., 1., 1., 1., 1., 1., 1.])}

In [ ]:
def evaluate(model, dataloader, device, map_metric, checkpoint_dir, epoch):
    model.eval()
    map_metric.reset()

    total_tp = 0
    total_fp = 0
    total_fn = 0
    iou_scores = []
    epoch_loss = 0

    with torch.no_grad():
        for images, annotations, targets, image_ids in dataloader:
            if images is None or annotations is None:
                continue

            # Move images and targets to device
            images = torch.stack([image.to(device) for image in images])
            annotations = {k: [t.to(device) for t in v] for k, v in annotations.items()}

            predictions = model(images)

            for pred, target in zip(predictions, targets):
                if pred["scores"].nelement() == 0:
                    continue

                # Update mAP metric
                map_metric.update([pred], [target])

                # Compute IoU between predicted and ground truth boxes
                pred_boxes = pred['boxes']
                gt_boxes = target['boxes']
                if pred_boxes.numel() == 0 or gt_boxes.numel() == 0:
                    continue

                iou_matrix = box_iou(pred_boxes, gt_boxes)
                max_iou_per_pred, matched_gt = iou_matrix.max(dim=1)

                # Define IoU threshold for TP/FP
                iou_threshold = 0.5
                tp = (max_iou_per_pred >= iou_threshold).sum().item()
                fp = (max_iou_per_pred < iou_threshold).sum().item()
                fn = max(0, len(gt_boxes) - tp)

                total_tp += tp
                total_fp += fp
                total_fn += fn

                # Save IoU scores to average later
                iou_scores.extend(max_iou_per_pred.cpu().tolist())

    # Compute mAP
    results = map_metric.compute()
    val_map = results['map'].item()
    print(f"\n📊 Object Detection Metrics:")
    print(f"Validation mAP: {val_map:.4f}")
    print(f"mAP@0.5: {results['map_50']:.4f}")
    print(f"mAP@0.75: {results['map_75']:.4f}")

    # Compute additional metrics
    # precision = total_tp / (total_tp + total_fp + 1e-6)
    # recall = total_tp / (total_tp + total_fn + 1e-6)
    # f1_score = 2 * precision * recall / (precision + recall + 1e-6)
    # mean_iou = sum(iou_scores) / len(iou_scores) if iou_scores else 0.0

    # print(f"Precision: {precision:.4f}")
    # print(f"Recall:    {recall:.4f}")
    # print(f"F1 Score:  {f1_score:.4f}")
    # print(f"Mean IoU:  {mean_iou:.4f}")

    # Save to file
    val_stats_path = os.path.join(checkpoint_dir, f"val_stats_{epoch + 1}.txt")
    with open(val_stats_path, "w") as f:
        f.write("Object Detection Metrics:\n")
        f.write(f"mAP@0.5: {results['map_50']:.4f}\n")
        f.write(f"mAP@0.75: {results['map_75']:.4f}\n")
        # f.write(f"Precision: {precision:.4f}\n")


In [ ]:
from effdet.bench import unwrap_bench


In [ ]:
# Training loop with model saving
num_epochs = 1
train_eval_steps = 10  # Evaluate on training set every 100 steps
step = 0
best_map = 0.0  # Track the best validation mAP

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    # Training phase
    for images, annotations, targets, image_ids in train_dataloader:
        step += 1

        # Skip empty batches
        # if images is None or targets is None:
        #     continue
        if images is None or annotations is None:
            continue

        # Move images and targets to device
        images = torch.stack([image.to(device) for image in images])
        annotations = {k: [t.to(device) for t in v] for k,v in annotations.items()}

        # Forward pass
        loss_dict = model(images, annotations)
        losses = loss_dict['loss']

        # Backward pass
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        epoch_loss += losses.item()

        # Evaluate on training set periodically
        if step % train_eval_steps == 0:

            print(f"{step}:{epoch_loss / (step+1)}")

    step = 0

    checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch + 1}.pth")

    checkpoint_path_dbt = os.path.join(checkpoint_dir_dbt, f"checkpoint_epoch_{epoch + 1}.pth")


    underlying_model = unwrap_bench(model=model)
    # print(underlying_model.state_dict())
    torch.save({
        "epoch": epoch + 1,
        #model.model refers to the weights for EfficientDet model w/o wrapper.
        "model_state_dict": underlying_model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "loss": epoch_loss / len(train_dataloader),
        # "val_map": val_stats["map"],
    }, checkpoint_path)
    print(f"Saved checkpoint at {checkpoint_path}")

    torch.save({
        "epoch": epoch + 1,
        #model.model refers to the weights for EfficientDet model w/o wrapper.
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "loss": epoch_loss / len(train_dataloader),
        # "val_map": val_stats["map"],
    }, checkpoint_path_dbt)
    print(f"Saved checkpoint at {checkpoint_path_dbt}")


    # try:
    #     evaluate(underlying_model, val_dataloader, device)
    # except:
    #     print("error evaluating")
    # evaluate(underlying_model, val_dataloader, device,map_metric=map_metric,checkpoint_dir=checkpoint_dir,epoch=epoch)
# def evaluate(model, dataloader, device, map_metric, checkpoint_dir, epoch):


10:5278.326332785867
20:3900.470211937314
30:3155.9017412739418
Saved checkpoint at ./checkpoints/effdet\checkpoint_epoch_1.pth
Saved checkpoint at ./checkpoints/effdet/dbt\checkpoint_epoch_1.pth


In [ ]:
#Load checkpoint
checkpoint_path = "./checkpoints/effdet/full/checkpoint_epoch_9.pth"

if not os.path.exists(checkpoint_path):
    raise FileNotFoundError(f"Checkpoint file not found at: {checkpoint_path}")
else:
    print("file exists")

checkpoint = torch.load(checkpoint_path)

file exists


In [ ]:
num_classes = 1
model_name = "efficientdet_d0"
image_size = (512,512)
config = get_efficientdet_config(model_name)
config.update({'num_classes': num_classes})
config.update({'image_size': image_size})

print(config)

net = EfficientDet(config, pretrained_backbone=False)
net.class_net = HeadNet(
    config,
    num_outputs=config.num_classes,
)


{'name': 'efficientdet_d0', 'backbone_name': 'efficientnet_b0', 'backbone_args': {'drop_path_rate': 0.1}, 'backbone_indices': None, 'image_size': [512, 512], 'num_classes': 1, 'min_level': 3, 'max_level': 7, 'num_levels': 5, 'num_scales': 3, 'aspect_ratios': [[1.0, 1.0], [1.4, 0.7], [0.7, 1.4]], 'anchor_scale': 4.0, 'pad_type': '', 'act_type': 'swish', 'norm_layer': None, 'norm_kwargs': {'eps': 0.001, 'momentum': 0.01}, 'box_class_repeats': 3, 'fpn_cell_repeats': 3, 'fpn_channels': 64, 'separable_conv': True, 'apply_resample_bn': True, 'conv_bn_relu_pattern': False, 'downsample_type': 'max', 'upsample_type': 'nearest', 'redundant_bias': False, 'head_bn_level_first': False, 'head_act_type': None, 'fpn_name': None, 'fpn_config': None, 'fpn_drop_path_rate': 0.0, 'alpha': 0.25, 'gamma': 1.5, 'label_smoothing': 0.0, 'legacy_focal': False, 'jit_loss': False, 'delta': 0.1, 'box_loss_weight': 50.0, 'soft_nms': False, 'max_detection_points': 5000, 'max_det_per_image': 100, 'url': 'https://githu

In [ ]:
net.load_state_dict(checkpoint['model_state_dict'])
net.to('cuda')
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
start_epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [ ]:
#Load checkpoint
checkpoint_path = "./checkpoints/effdet/dbt/full/checkpoint_epoch_9.pth"

if not os.path.exists(checkpoint_path):
    raise FileNotFoundError(f"Checkpoint file not found at: {checkpoint_path}")
else:
    print("file exists")

checkpoint = torch.load(checkpoint_path)

file exists


In [ ]:
num_classes = 1
model_name = "efficientdet_d0"
image_size = (512,512)
config = get_efficientdet_config(model_name)
config.update({'num_classes': num_classes})
config.update({'image_size': image_size})

print(config)

net = EfficientDet(config, pretrained_backbone=False)
net.class_net = HeadNet(
    config,
    num_outputs=config.num_classes,
)
model = DetBenchTrain(net, config)

{'name': 'efficientdet_d0', 'backbone_name': 'efficientnet_b0', 'backbone_args': {'drop_path_rate': 0.1}, 'backbone_indices': None, 'image_size': [512, 512], 'num_classes': 1, 'min_level': 3, 'max_level': 7, 'num_levels': 5, 'num_scales': 3, 'aspect_ratios': [[1.0, 1.0], [1.4, 0.7], [0.7, 1.4]], 'anchor_scale': 4.0, 'pad_type': '', 'act_type': 'swish', 'norm_layer': None, 'norm_kwargs': {'eps': 0.001, 'momentum': 0.01}, 'box_class_repeats': 3, 'fpn_cell_repeats': 3, 'fpn_channels': 64, 'separable_conv': True, 'apply_resample_bn': True, 'conv_bn_relu_pattern': False, 'downsample_type': 'max', 'upsample_type': 'nearest', 'redundant_bias': False, 'head_bn_level_first': False, 'head_act_type': None, 'fpn_name': None, 'fpn_config': None, 'fpn_drop_path_rate': 0.0, 'alpha': 0.25, 'gamma': 1.5, 'label_smoothing': 0.0, 'legacy_focal': False, 'jit_loss': False, 'delta': 0.1, 'box_loss_weight': 50.0, 'soft_nms': False, 'max_detection_points': 5000, 'max_det_per_image': 100, 'url': 'https://githu

In [ ]:
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
# Training loop with model saving
total_epochs = 15
train_eval_steps = 10  # Evaluate on training set every 100 steps
step = 0
best_map = 0.0  # Track the best validation mAP


for epoch in range(start_epoch,total_epochs):
    model.train()
    epoch_loss = 0

    # Training phase
    for images, annotations, targets, image_ids in train_dataloader:
        step += 1

        # Skip empty batches
        # if images is None or targets is None:
        #     continue
        if images is None or annotations is None:
            continue

        # Move images and targets to device
        images = torch.stack([image.to(device) for image in images])
        annotations = {k: [t.to(device) for t in v] for k,v in annotations.items()}

        # Forward pass
        loss_dict = model(images, annotations)
        losses = loss_dict['loss']

        # Backward pass
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        epoch_loss += losses.item()

        # Evaluate on training set periodically
        if step % train_eval_steps == 0:

            print(f"{step}:{epoch_loss / (step+1)}")

    step = 0

    checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch + 1}.pth")

    checkpoint_path_dbt = os.path.join(checkpoint_dir_dbt, f"checkpoint_epoch_{epoch + 1}.pth")


    underlying_model = unwrap_bench(model=model)
    # print(underlying_model.state_dict())
    torch.save({
        "epoch": epoch + 1,
        #model.model refers to the weights for EfficientDet model w/o wrapper.
        "model_state_dict": underlying_model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "loss": epoch_loss / len(train_dataloader),
        # "val_map": val_stats["map"],
    }, checkpoint_path)
    print(f"Saved checkpoint at {checkpoint_path}")

    torch.save({
        "epoch": epoch + 1,
        #model.model refers to the weights for EfficientDet model w/o wrapper.
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "loss": epoch_loss / len(train_dataloader),
        # "val_map": val_stats["map"],
    }, checkpoint_path_dbt)
    print(f"Saved checkpoint at {checkpoint_path_dbt}")

In [ ]:
eff_det_model(sample_images)

([tensor([[[[-0.8579, -0.8839, -1.3323,  ..., -1.2024, -1.0493, -0.8084],
            [-1.0886, -1.1546, -1.4077,  ..., -1.3872, -1.2839, -1.3937],
            [-1.1435, -1.3630, -1.3746,  ..., -1.3872, -1.3233, -1.3440],
            ...,
            [-1.0510, -1.3314, -1.1683,  ..., -1.4262, -1.3461, -1.1178],
            [-0.9221, -1.4022, -1.0572,  ..., -1.1019, -1.0922, -0.8975],
            [-0.5441, -1.1848, -0.8363,  ..., -0.9805, -0.8609, -0.5197]],
  
           [[-0.4900, -0.5657, -1.2587,  ..., -1.3788, -1.0532, -0.8162],
            [-0.7816, -0.9715, -1.3897,  ..., -1.5014, -1.2816, -1.0555],
            [-0.8176, -0.9918, -1.4135,  ..., -1.5242, -1.2512, -1.1352],
            ...,
            [-1.0282, -1.1231, -1.1176,  ..., -1.2205, -1.0107, -0.8010],
            [-0.9837, -1.1768, -1.0133,  ..., -1.0131, -1.0143, -0.7574],
            [-0.8808, -1.2965, -1.1093,  ..., -1.0566, -0.9188, -0.5530]],
  
           [[-0.4794, -0.3898, -0.7198,  ..., -0.5031, -0.4238, -0.263

In [ ]:
sample_images = sample_batch[0]
sample_annotations = sample_batch[1]

In [ ]:
model.to("cpu")
model.eval()
output = model(sample_images,sample_annotations)
output['detections'].shape

torch.Size([8, 100, 6])